In [1]:
import numpy as np
import random
from proglearn.sims import generate_gaussian_parity
import matplotlib.pyplot as plt

# k sample testing from hyppo
from hyppo.ksample import KSample
from hyppo.tools import rot_ksamp

from proglearn.forest import LifelongClassificationForest, UncertaintyForest
from proglearn.sims import *
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.deciders import SimpleArgmaxAverage
from proglearn.transformers import (
    TreeClassificationTransformer,
    NeuralClassificationTransformer,
)
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter

from joblib import Parallel, delayed

from math import log2, ceil

In [2]:
# define angle
angle = np.pi / 4
# samples to use per distribution
n_samples = 1000
# generate xor
X_xor, y_xor = generate_gaussian_parity(n_samples)
# generate most adversarial rxor
X_rxor, y_rxor = generate_gaussian_parity(n_samples, angle_params=angle)

In [3]:
# define number of estimators to use
n_trees = 10

# train single run of model
progressive_learner = LifelongClassificationForest(default_n_estimators=n_trees)
# add xor data
progressive_learner.add_task(X_xor, y_xor, n_estimators=n_trees)
# k sample test, add new task rxor and learn new transformer if p <= 0.05 
if KSample(indep_test="Dcorr").test(X_xor, X_rxor)[1] <= 0.05:
    progressive_learner.add_task(X_rxor, y_rxor, n_estimators=n_trees)
    progressive_learner.add_transformer(X_rxor, y_rxor, n_estimators=n_trees)
else:
    # else add concatenated xor and rxor data 
    X_xor_rxor = np.concatenate((X_xor, X_rxor), axis=0)
    y_xor_rxor = np.concatenate((y_xor, y_rxor), axis=0)
    progressive_learner.add_task(X_xor_rxor, y_xor_rxor, n_estimators=n_trees)
